In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
import tensorflow as tf
import time

In [ ]:
# Wczytaj
# - train.csv
# - model.h5

# Wczytanie modelu glownego

In [ ]:
model = tf.keras.models.load_model('model.h5')
teacher_model = model

# Tworzenie modelu naucyzciela

In [ ]:
datagen = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            rotation_range=10,
            zoom_range = 0.1,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=False,
            vertical_flip=False)

In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last',
                     input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

# Wczytanie danych

In [ ]:
data = pd.read_csv('train.csv')

X = data.iloc[:,1:]
Y = data.iloc[:,0]


X_reshaped = X.values.reshape(-1, 28, 28, 1)

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(X_reshaped, Y, test_size=0.1)

x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255

datagen.fit(x_train)

In [ ]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [ ]:
def soft_labels(model, x, temperature):
    """
    Generuje miękkie etykiety z modelu nauczyciela przy podwyższonej temperaturze.
    """
    logits = model.predict(x)
    logits /= temperature
    return np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.models import save_model
from tensorflow.keras.losses import KLDivergence

# Lista modeli ucznia do przechowywania wytrenowanych modeli
student_models = []

T_list = [40, 50]

for T in T_list:
    print(f"\nTrenowanie modelu ucznia z T={T}")

    # Generowanie miękkich etykiet dla danych treningowych i testowych
    soft_y_train = soft_labels(teacher_model, x_train, T)
    soft_y_test = soft_labels(teacher_model, x_test, T)

    # Tworzenie nowego modelu ucznia
    student_model = create_model()

    # Learning Rate Scheduler
    reduce_lr = LearningRateScheduler(lambda epoch: 1e-3 * 0.9 ** epoch)

    # Early Stopping
    early_stopping = EarlyStopping(
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        restore_best_weights=True
    )

    # Kompilacja modelu ucznia
    student_model.compile(optimizer=Adam(), loss=KLDivergence(), metrics=["accuracy"])

    # Trening modelu pomocniczego
    student_model.fit(
        datagen.flow(x_train, y_train, batch_size=64),
        epochs=50,
        validation_data=(x_test, y_test),
        verbose=1,
        steps_per_epoch=x_train.shape[0] // 64,
        callbacks=[reduce_lr, early_stopping]
    )

    # Zapis modelu do listy
    student_models.append(student_model)

    # Zapis modelu na dysk
    model_name = f"DD_model_T{T}.h5"
    save_model(student_model, model_name)
    print(f"Model zapisany jako {model_name}")


Trenowanie modelu ucznia z T=40
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


590/590 ━━━━━━━━━━━━━━━━━━━━ 32s 39ms/step - accuracy: 0.7880 - loss: 0.7447 - val_accuracy: 0.9698 - val_loss: 0.1024 - learning_rate: 0.0010
Epoch 2/50
  1/590 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9531 - loss: 0.0870

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 540us/step - accuracy: 0.9531 - loss: 0.0870 - val_accuracy: 0.9700 - val_loss: 0.1023 - learning_rate: 9.0000e-04
Epoch 3/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.9596 - loss: 0.1353 - val_accuracy: 0.9836 - val_loss: 0.0520 - learning_rate: 8.1000e-04
Epoch 4/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - accuracy: 0.9219 - loss: 0.1069 - val_accuracy: 0.9831 - val_loss: 0.0520 - learning_rate: 7.2900e-04
Epoch 5/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.9695 - loss: 0.0969 - val_accuracy: 0.9883 - val_loss: 0.0374 - learning_rate: 6.5610e-04
Epoch 6/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step - accuracy: 0.9219 - loss: 0.2343 - val_accuracy: 0.9879 - val_loss: 0.0376 - learning_rate: 5.9049e-04
Epoch 7/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.9757 - loss: 0.0750 - val_accuracy: 0.9912 - val_loss: 0.0319 - learning_rate: 5.3144e-04
Epoch 8/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - accurac

Model zapisany jako DD_model_T40.h5

Trenowanie modelu ucznia z T=50
1182/1182 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 30s 37ms/step - accuracy: 0.7882 - loss: 0.7585 - val_accuracy: 0.9614 - val_loss: 0.1201 - learning_rate: 0.0010
Epoch 2/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step - accuracy: 0.9688 - loss: 0.0570 - val_accuracy: 0.9612 - val_loss: 0.1164 - learning_rate: 9.0000e-04
Epoch 3/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 31s 27ms/step - accuracy: 0.9557 - loss: 0.1420 - val_accuracy: 0.9814 - val_loss: 0.0689 - learning_rate: 8.1000e-04
Epoch 4/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.9375 - loss: 0.2047 - val_accuracy: 0.9807 - val_loss: 0.0697 - learning_rate: 7.2900e-04
Epoch 5/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - accuracy: 0.9685 - loss: 0.0955 - val_accuracy: 0.9869 - val_loss: 0.0450 - learning_rate: 6.5610e-04
Epoch 6/50
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step - accur

Model zapisany jako DD_model_T50.h5
